In [15]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import json
from urllib2 import urlopen, Request
from bs4 import BeautifulSoup


view_party=pd.read_csv('../refdata/view_party.csv').drop(['cmp','euprofiler','ees','castles_mair','huber_inglehart','ray','benoit_laver','chess'],axis=1)
partyColumns=view_party.columns

view_election=pd.read_csv('../refdata/view_election.csv')
electionColumns=view_election.columns

view_election=view_election.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_election=view_election.drop([x for x in view_election.columns if x.endswith('_2')],axis=1)
view_election.election_date=view_election.election_date.apply(dateutil.parser.parse)

view_cabinet=pd.read_csv('../refdata/view_cabinet.csv')
cabinetColumns=view_cabinet.columns

view_cabinet=view_cabinet.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_cabinet=view_cabinet.drop([x for x in view_cabinet.columns if x.endswith('_2')],axis=1)
view_cabinet=view_cabinet.merge(view_election,on=['election_id','party_id'],how='left',suffixes=('','_2'))
view_cabinet=view_cabinet.drop([x for x in view_cabinet.columns if x.endswith('_2')],axis=1)
view_cabinet.election_date=view_cabinet.election_date.apply(dateutil.parser.parse)


NationalElections=view_election[view_election.election_type=='parliament']
EuroElections=view_election[view_election.election_type=='ep']

def getLastElection(t):
    return t[t.election_date==t.election_date.max()]

def getLastCabinet(t):
    return t[t.start_date==t.start_date.max()]

NationalElections=NationalElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
EuroElections=EuroElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
Cabinets=view_cabinet.groupby('country_name_short').apply(getLastCabinet).reset_index(drop=True)

NationalElections=NationalElections[~NationalElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
EuroElections=EuroElections[~EuroElections.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])]
Cabinets=Cabinets[~Cabinets.country_name_short.isin(['AUS','CAN','ISR','NZL','JPN'])].drop(['election_type','previous_cabinet_id','previous_parliament_election_id'],axis=1)

NationalElections.to_csv('../refdata/national_elections.csv',index=False)
EuroElections.to_csv('../refdata/euro_elections.csv',index=False)
Cabinets.to_csv('../refdata/cabinets.csv',index=False)

CPI_2015_data=pd.read_excel('../build/CPI_2015_data.xlsx')
CPI_2015_data=CPI_2015_data[CPI_2015_data.Region.isin(['ECA','WE/EU'])]
CPI_2015_data.loc[CPI_2015_data[CPI_2015_data.wbcode=='ROM'].index,'wbcode']='ROU'
CPI_2015_data.loc[CPI_2015_data[CPI_2015_data.wbcode=='MON'].index,'wbcode']='MNE'
CPI_2015_data.loc[CPI_2015_data[CPI_2015_data.wbcode=='SCG'].index,'wbcode']='SRB'
CPI_2015_data.loc[CPI_2015_data[CPI_2015_data.wbcode=='LWI'].index,'wbcode']='KOS'

WDI_data=pd.read_csv('../build/WDI_data.csv')
WDI_data.loc[WDI_data[WDI_data['Country Code']=='ROM'].index,'Country Code']='ROU'
WDI_data_2015=WDI_data[['Country Code','Indicator Code','2015']][WDI_data['Country Code'].isin(CPI_2015_data.wbcode)] \
    .set_index(['Country Code','Indicator Code']).unstack()['2015'][['SP.POP.TOTL','NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD','EN.POP.DNST']] \
    .reset_index().rename(columns={
            'Country Code':'countryCode',
            'SP.POP.TOTL':'totalPopulation',
            'NY.GDP.MKTP.CD':'gdpUsd',
            'NY.GDP.PCAP.CD':'gdpPerCapitaUsd',
            'EN.POP.DNST' : 'populationDensity',
        }).rename_axis(None,axis=1)
    
WDI_data_2014=WDI_data[['Country Code','Indicator Code','2014']][WDI_data['Country Code'].isin(CPI_2015_data.wbcode)] \
    .set_index(['Country Code','Indicator Code']).unstack()['2014'][['SL.UEM.TOTL.ZS','SL.UEM.TOTL.NE.ZS']] \
    .reset_index().rename(columns={
            'Country Code':'countryCode',
            'SL.UEM.TOTL.ZS' : 'unemploymentIlo',
            'SL.UEM.TOTL.NE.ZS' : 'unemploymentNat'
        }).rename_axis(None,axis=1)
    
WDI_data=WDI_data_2015.merge(WDI_data_2014,how='left')
    
Country=pd.read_csv('../refdata/country.csv')[['name_short','oecd_accession_date','eu_accession_date']]

bs = BeautifulSoup(urlopen(Request('https://www.cia.gov/library/publications/the-world-factbook/fields/2128.html')))
trs=bs.find(id='fieldListing').findAll('tr')
GovernmentType=DataFrame([{'id': tr.get('id'), 'data': tr.find(class_='fieldData').getText().strip()} for tr in trs if not tr.get('class')])
GovernmentType['fips']=GovernmentType.id.str.upper()
GovernmentType=GovernmentType.drop('id',axis=1)

ExternalCountry=pd.read_csv('../refdata/external_country_iso.csv')
ExternalCountry.loc[ExternalCountry[ExternalCountry.iso3=='ROM'].index,'iso3']='ROU'
ExternalCountry=pd.concat([ExternalCountry,DataFrame([{'isonumeric': 688, 'country': 'Serbia', 'region': 'South East Europe', 'capital': 'Belgrade', 'id': 688, 'iso3': 'SRB', 'iso2': 'RS', 'fips': 'RI', 'internet': 'RS', 'continent': 'Europe'},
    {'isonumeric': 499, 'country': 'Montenegro', 'region': 'South East Europe', 'capital': 'Podgorica', 'id': 499, 'iso3': 'MNE', 'iso2': 'ME', 'fips': 'MJ', 'internet': 'ME', 'continent': 'Europe'},
    {'isonumeric': 780, 'country': 'Kosovo', 'region': 'South East Europe', 'capital': 'Prishtina', 'id': 780, 'iso3': 'KOS', 'iso2': 'KO', 'fips': 'KV', 'internet': 'KS', 'continent': 'Europe'}])],ignore_index=True)

ExternalCountry=ExternalCountry[ExternalCountry.iso3.isin(CPI_2015_data.wbcode)].reset_index()
ExternalCountry=ExternalCountry.merge(GovernmentType,how='left',on='fips').rename(columns={'data':'governmentType'})

def getPartyJson(code):
    r = {}
        
    if code in Cabinets.country_name_short.tolist():
        topLevelFields=['country_id','country_name_short','country_name','election_id','election_date','seats_total',
                   'cabinet_id','cabinet_name','start_date','caretaker','election_seats_total']
        nat = Cabinets[code==Cabinets.country_name_short] \
            .sort_values(['seats','vote_share','party_name_short'],ascending=False)
        nat.seats_total=nat.seats_total.fillna(-1)
        t=nat[topLevelFields].reset_index(drop=True).ix[0].to_dict()
        t['election_date']=str(t['election_date'])
        r.update(t)
        r.update({'parties' : [x[1][nat.columns.difference(topLevelFields)].dropna().to_dict() for x in nat.iterrows()]})    
    if code in CPI_2015_data.wbcode.tolist():
        r['cpi2015'] = CPI_2015_data[CPI_2015_data.wbcode==code].CPI2015.tolist()[0]
    if code in WDI_data.countryCode.tolist():
        r.update(WDI_data[WDI_data.countryCode==code].reset_index(drop=True).T.drop('countryCode')[0].dropna().to_dict())
    if code in Country.name_short.tolist():        
        r.update(Country[Country.name_short==code].reset_index(drop=True).T.drop('name_short')[0].dropna().to_dict())
    if code in ExternalCountry.iso3.tolist():        
        r.update(ExternalCountry[ExternalCountry.iso3==code].reset_index(drop=True).T.drop(['iso3','id','index','country'])[0].dropna().to_dict())        
    return r

data={code:getPartyJson(code) for code in  CPI_2015_data.wbcode.drop_duplicates().tolist() if not code in ['AUS','CAN','ISR','NZL','JPN']}

with open('../public/parlgov.json', 'w') as outfile:
    json.dump(data, outfile, default='')

In [17]:
print CPI_2015_data.wbcode.drop_duplicates().tolist()
print WDI_data.countryCode.tolist()

[u'DNK', u'FIN', u'SWE', u'NLD', u'NOR', u'CHE', u'DEU', u'LUX', u'GBR', u'ISL', u'BEL', u'AUT', u'IRL', u'EST', u'FRA', u'PRT', u'POL', u'CYP', u'LTU', u'SVN', u'ESP', u'CZE', u'MLT', u'LVA', u'GEO', u'HRV', u'HUN', u'SVK', u'GRC', 'ROU', u'ITA', 'MNE', u'MKD', u'TUR', u'BGR', 'SRB', u'BIH', u'ALB', u'ARM', 'KOS', u'MDA', u'BLR', u'AZE', u'RUS', u'KAZ', u'KGZ', u'UKR', u'TJK', u'UZB', u'TKM']
['ALB', 'ARM', 'AUT', 'AZE', 'BEL', 'BGR', 'BIH', 'BLR', 'CHE', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GEO', 'GRC', 'HRV', 'HUN', 'IRL', 'ISL', 'ITA', 'KAZ', 'KGZ', 'LTU', 'LUX', 'LVA', 'MDA', 'MKD', 'MLT', 'MNE', 'NLD', 'NOR', 'POL', 'PRT', 'RUS', 'SRB', 'SVK', 'SVN', 'SWE', 'TJK', 'TKM', 'TUR', 'UKR', 'UZB']


In [37]:
set([([party.get('family_name') for party in data[x]['parties'] if party.get('cabinet_party')][0]) for x in data if 'parties' in data[x]])

{'Agrarian',
 'Christian democracy',
 'Communist/Socialist',
 'Conservative',
 'Liberal',
 'Social democracy',
 'no family'}

In [27]:
(lambda x: (x['mean']+ x['std']*pd.Series([-1,0,1])).tolist())(pd.Series(sorted([data[x].get('unemploymentIlo') for x in data])).describe())

[4.4261262095529217, 10.802040868875935, 17.177955528198947]

In [13]:
CPI_2015_data.wbcode.sort_values()
WDI_data_2015

,countryCode,totalPopulation,gdpUsd,gdpPerCapitaUsd,populationDensity
0,ALB,2889167.0,1.145560e+10,3965.016806,105.444051
1,ARM,3017712.0,1.056140e+10,3499.804218,105.996207
2,AUT,8611088.0,3.740559e+11,43438.863038,104.337619
3,AZE,9651349.0,5.304714e+10,5496.344640,116.761018
4,BEL,11285721.0,4.540390e+11,40231.283174,372.712054
5,BGR,7177991.0,4.895296e+10,6819.869108,66.120035
6,BIH,3810416.0,1.599539e+10,4197.807304,74.422188
7,BLR,9513000.0,5.460896e+10,5740.456495,46.882854
8,CHE,8286976.0,6.647375e+11,80214.730152,209.711914
9,CYP,1165300.0,1.931973e+10,22957.397983,126.114719


In [116]:
code='SRB'

print 'CHE'in ExternalCountry.iso3.tolist()

ExternalCountry[ExternalCountry.iso3==code].reset_index(drop=True).T.drop(['iso3','id','index','country'])[0].dropna().to_dict()

True


{'capital': 'Belgrade',
 'continent': 'Europe',
 'data': u'parliamentary republic',
 'fips': 'RI',
 'internet': 'RS',
 'iso2': 'RS',
 'isonumeric': 688,
 'region': 'South East Europe'}

In [10]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser
import json
CPI_2015_data=pd.read_excel('../build/CPI_2015_data.xlsx')
CPI_2015_data=CPI_2015_data[CPI_2015_data.Region.isin(['ECA','WE/EU'])]
CPI_2015_data.loc[CPI_2015_data[CPI_2015_data.wbcode=='ROM'].index,'wbcode']='ROU'

CPI_2015_data

,Rank,CPI2015,Country,Region,wbcode,World Bank CPIA,World Economic Forum EOS,Bertelsmann Foundation TI,African Dev Bank,IMD World Competitiveness Yearbook,...,CPI2015(2),Rank2,Number of Sources,Std Deviation of Sources,Standard Error,Minimum,Maximum,Lower CI,Upper CI,Country2
0,1,91,Denmark,WE/EU,DNK,NaN,89.0,NaN,NaN,96.0,...,91,1,7,5.71,2.16,83,98,87.4360,94.5640,Denmark
1,2,90,Finland,WE/EU,FIN,NaN,93.0,NaN,NaN,91.0,...,90,2,7,4.69,1.77,83,98,87.0795,92.9205,Finland
2,3,89,Sweden,WE/EU,SWE,NaN,89.0,NaN,NaN,87.0,...,89,3,7,4.53,1.71,83,98,86.1785,91.8215,Sweden
4,5,87,Netherlands,WE/EU,NLD,NaN,84.0,NaN,NaN,85.0,...,87,5,7,4.79,1.81,83,97,84.0135,89.9865,Netherlands
5,5,87,Norway,WE/EU,NOR,NaN,92.0,NaN,NaN,84.0,...,87,5,7,7.93,3.00,73,98,82.0500,91.9500,Norway
6,7,86,Switzerland,WE/EU,CHE,NaN,87.0,NaN,NaN,88.0,...,86,7,6,6.25,2.55,73,89,81.7925,90.2075,Switzerland
9,10,81,Germany,WE/EU,DEU,NaN,72.0,NaN,NaN,83.0,...,81,10,7,6.62,2.50,72,89,76.8750,85.1250,Germany
10,10,81,Luxembourg,WE/EU,LUX,NaN,89.0,NaN,NaN,85.0,...,81,10,5,13.45,6.02,57,89,71.0670,90.9330,Luxembourg
11,10,81,United Kingdom,WE/EU,GBR,NaN,82.0,NaN,NaN,81.0,...,81,10,7,6.36,2.40,73,89,77.0400,84.9600,United Kingdom
13,13,79,Iceland,WE/EU,ISL,NaN,87.0,NaN,NaN,83.0,...,79,13,5,10.14,4.53,65,89,71.5255,86.4745,Iceland


In [14]:
CPI_2015_data=pd.read_excel('../build/CPI_2015_data.xlsx')
CPI_2015_data=CPI_2015_data[CPI_2015_data.Region.isin(['ECA','WE/EU'])]

WDI_data=pd.read_csv('../build/WDI_data.csv')
WDI_data=WDI_data[['Country Code','Indicator Code','2015']][WDI_data['Country Code'].isin(CPI_2015_data.wbcode)] \
    .set_index(['Country Code','Indicator Code']).unstack()['2015'][['SP.POP.TOTL','NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD']] \
    .reset_index().rename(columns={
            'Country Code':'countryCode',
            'SP.POP.TOTL':'totalPopulation',
            'NY.GDP.MKTP.CD':'gdpUsd',
            'NY.GDP.PCAP.CD':'gdpPerCapitaUsd',
        }).rename_axis(None,axis=1)

In [15]:
WDI_data

,countryCode,totalPopulation,gdpUsd,gdpPerCapitaUsd
0,ALB,2889167.0,1.145560e+10,3965.016806
1,ARM,3017712.0,1.056140e+10,3499.804218
2,AUT,8611088.0,3.740559e+11,43438.863038
3,AZE,9651349.0,5.304714e+10,5496.344640
4,BEL,11285721.0,4.540390e+11,40231.283174
5,BGR,7177991.0,4.895296e+10,6819.869108
6,BIH,3810416.0,1.599539e+10,4197.807304
7,BLR,9513000.0,5.460896e+10,5740.456495
8,CHE,8286976.0,6.647375e+11,80214.730152
9,CYP,1165300.0,1.931973e+10,22957.397983


In [68]:
code='AUT'

if code in CPI_2015_data.wbcode:
    print 1

WDI_data[WDI_data.countryCode==code].reset_index(drop=True).T.drop('countryCode')[0].to_dict()

{'gdpPerCapitaUsd': 43438.8630381343,
 'gdpUsd': 374055872241.322,
 'totalPopulation': 8611088.0}

In [79]:
%%bash
ls ../refdata/*.csv

../refdata/cabinets.csv
../refdata/country.csv
../refdata/euro_elections.csv
../refdata/europe_country_doc.csv
../refdata/europe_election_doc.csv
../refdata/europe_party_doc.csv
../refdata/external_country_iso.csv
../refdata/national_elections.csv
../refdata/politician_president.csv
../refdata/view_cabinet.csv
../refdata/view_election.csv
../refdata/view_party.csv


In [29]:
Cabinets[Cabinets.country_name=='Romania']

,country_name_short,country_name,election_date,start_date,cabinet_name,caretaker,cabinet_party,prime_minister,seats,election_seats_total,...,party_id,party_name_ascii,family_name_short,family_name,state_market,liberty_authority,eu_anti_pro,family_id,vote_share,seats_total
276,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,22.0,412,...,2401.0,Partidul Liberal Reformator,lib,Liberal,6.7000,3.5000,8.7000,6.0,NaN,NaN
277,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,13.0,412,...,5.0,Partidul Conservator,con,Conservative,4.7538,6.2779,8.2028,26.0,NaN,412.0
278,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,56.0,412,...,958.0,Partidul Democrat-Liberal,lib,Liberal,5.1868,3.9647,9.5309,6.0,16.51,412.0
279,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,78.0,412,...,1015.0,Partidul National Liberal,lib,Liberal,7.2450,3.5474,9.6079,6.0,NaN,412.0
280,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,47.0,412,...,2130.0,Partidul Poporului -- Dan Diaconescu,com,Communist/Socialist,1.4000,3.0000,3.3000,14.0,13.99,412.0
281,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,160.0,412,...,1120.0,Partidul Social Democrat,soc,Social democracy,2.4518,6.2794,8.6459,11.0,58.63,412.0
282,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,18.0,412,...,948.0,Uniunea Democratica Maghiara din Romania,con,Conservative,5.9825,3.8813,9.4801,26.0,5.13,412.0
283,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,0,0,18.0,412,...,425.0,Partidele minoritatilor etnice din Romania,none,no family,NaN,NaN,NaN,12.0,NaN,412.0
284,ROU,Romania,2012-12-09,2015-11-10,Ciolos,1,1,1,0.0,412,...,761.0,no party affiliation,none,no family,NaN,NaN,NaN,12.0,NaN,NaN


In [ ]:
PoliticianPresident=pd.read_csv('../refdata/politician_president.csv')
#ExternalCountry=ExternalCountry[(ExternalCountry.continent=='Europe')|(ExternalCountry.iso3.isin(['RUS','CYP','CYN','TUR']))]
PoliticianPresident.start_date=PoliticianPresident.start_date.apply(dateutil.parser.parse)

PoliticianPresident=PoliticianPresident[PoliticianPresident.end_date.isnull()].sort_values('start_date')
PoliticianPresident
Cabinets.merge(PoliticianPresident,on='country_id',how='left',suffixes=('_','')).merge(Cabinets,on='party_id',how='left',suffixes=('_',''))[['country_name','person_id_source','party_id','party_name']].drop_duplicates()

In [103]:

  #  , headers={'User-Agent': useragent}
bs = BeautifulSoup(urlopen(Request('https://www.cia.gov/library/publications/the-world-factbook/fields/2128.html')))
trs=bs.find(id='fieldListing').findAll('tr')
GovernmentType=DataFrame([{'id': tr.get('id'), 'data': tr.find(class_='fieldData').getText().strip()} for tr in trs if not tr.get('class')])
GovernmentType['fips']=GovernmentType.id.str.upper()
GovernmentType=GovernmentType.drop('id',axis=1)